In [2]:
import sys
sys.path.append("../../")

In [3]:
from facenet_pytorch import MTCNN

In [16]:
from torch.utils import data
from torch import optim
import torch
from torch import nn
from torch.optim import lr_scheduler

In [17]:
from glob import glob 
import json
import pandas
import typing
import os

In [18]:
from src.training.losses import losses
from src.training.metrics import metrics as eval_metrics
from src.training.trainers import regularization
from src.preprocessing import augmentations

In [7]:
TRAIN_DIR = "../experiments/experiment1/data/train_data"
TRAIN_ANNOTATIONS = "../experiments/experiment1/labels/train_labels"
VALIDATION_DIR = "../experiments/experiment1/data/validation_data"
VALIDATION_ANNOTATIONS = "../experiments/experiment1/labels/validation_labels"
DATA_CONFIG_DIR = "../experiments/experiment1/data/"

In [ ]:
class XMLAnnotationParser(object):
    """
    Class for extracting annotations
    from .xml files.
    """
    def parse_annotations(self, input_path: str) -> pandas.DataFrame:
        pass

In [7]:
def load_xml_annotations(annotations_path: str) -> pandas.DataFrame:
    """
    Loads xml annotations from the specified 
    'annotations_path' directory folder.

    Parameters:
    -----------
        annotations_path - path, containing xml annotations for data
    """
    parser = XMLAnnotationParser()
    output_anns = None 
    
    for path in glob(pathname="**/*.%s" % file_ext, root_dir=annotations_path):
        ann_path = os.path.join(annotations_path, path)
        
        annotations = parser.parse_annotations(ann_path)
        if output_anns is None:
            output_anns = annotations
        else:
            output_anns = pandas.concat([output_anns, annotations], axis=0)
    return output_anns

In [8]:
def load_image_data(root_dir: str, file_extensions: typing.List):
    output_images = []
    
    for ext in file_extensions:
        found_images = glob(pathname="**/*.%s" % ext, root_dir=root_dir)
        output_images.extend(found_images)
        
    return output_images

# Loading annotations for training and validation data

In [9]:
train_annotations = load_xml_annotations(TRAIN_ANNOTATIONS)
validation_annotations = load_xml_annotations(VALIDATION_ANNOTATIONS)

# Loading image train and validation datasets

In [ ]:
train_dataset = load_image_data(TRAIN_DATASET_DIR, ["jpeg", "png", "jpg"])
validation_dataset = load_image_data(VALIDATION_DATASET_DIR, ["jpeg", "png", "jpg"])

In [ ]:
train_hashes = []
train_annotations.set_index("video_name")
sorted_train_annotations = train_annotations.reindex(train_hashes)

In [ ]:
validation_hashes = []
validation_annotations.set_index("video_name")
sorted_validation_annotations = validation_annotations.reindex(validation_hashes)

# Augmentations

In [ ]:
train_augmentations = augmentations.get_train_augmentations(MTCNN_IMAGE_SIZE)
validation_augmentations = augmentations.get_valid_augmentations(MTCNN_IMAGE_SIZE)

# Initializing datasets

In [ ]:
%%time

train_dataset = datasets.MTCNNFineTuneDataset(
    image_paths=train_dataset,
    boxes=sorted_train_annotations['boxes'].tolist(),
    transformations=train_augmentations
)

validation_dataset = datasets.MTCNNFineTuneDataset(
    image_paths=validation_dataset,
    boxes=sorted_validation_annotations['boxes'].tolist(),
    transformations=validation_augmentations
)

early_stop_dataset = datasets.MTCNNFineTuneDataset(
    image_paths=early_dataset,
    boxes=early_annotations['boxes'].tolist(),
    transformations=validation_augmentations
)

# Initializing configuration

In [14]:
max_epochs = 20
learning_rate = 1e-5
weight_decay = 0.02

if torch.cuda.is_available():
    device = torch.device("cpu")
else:
    device = torch.device("cpu")

# early stopping patience

early_patience = 5
min_diff = 0.05
early_start = 2
early_stopper = regularization.EarlyStopping(early_patience, min_diff)

# initialization data loaders

workers_per_loader = max(os.cpu_count()-1, 0) // 3 # for train, val and early loaders


In [22]:
# eval metric and loss function for fine-tuning
loss_function = losses.CIOULoss()
eval_metric = eval_metrics.IOUScore()

# network, optimizer and LR Scheduling techniques

network = MTCNN(
    min_face_size=160, margin=10, 
    post_process=False, 
    thresholds=[0.8, 0.9, 0.95],
    device=device
)

optimizer = optim.Adam(network.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = lr_scheduler.MultiStepLR(optimizer, [5, 10, 15, 20], gamma=0.6)

In [13]:
%%time

train_loader = data.DataLoader(
    dataset=train_dataset,
    shuffle=True,
    num_workers=workers_per_loader,
    batch_size=train_batch_size
)

val_loader = data.DataLoader(
    batch_size=1,
    dataset=validation_dataset,
    shuffle=True,
    num_workers=workers_per_loader
)

early_loader = data.DataLoader(
    batch_size=1,
    dataset=early_dataset,
    shuffle=True,
    num_workers=workers_per_loader
)

NameError: name 'train_dataset' is not defined

In [23]:
%%time

class TunePipeline(object):
    """
    Training pipeline for tuning
    MTCNN face detector
    """
    def __init__(self,
        network: nn.Module,
        loss_function: nn.Module,
        eval_metric: nn.Module,
        max_epochs: int,
        batch_size: int,
        optimizer: nn.Module,
        lr_scheduler: nn.Module,
        snapshot_path: str,
        inf_device
    ):
        self.network = network.to(device)
        self.loss_function = loss_function
        self.eval_metric = eval_metric
        self.max_epochs: int = max_epochs
        self.batch_size: int = batch_size
        self.optimizer = optimizer
        self.lr_scheduler = lr_scheduler
        self.snapshot_path = snapshot_path
        self.inf_device = inf_device

    def save_checkpoint(self, epoch: int, loss: float):
        
        snapshot_full_path = os.path.join(
            self.snapshot_path, 
            "model_epoch_%s.pth" % epoch
        )
        
        snapshot = {
            'model_state': self.network.state_dict(),
            'optimizer_state': self.optimizer.state_dict(),
            'lr_scheduler_state': self.lr_scheduler.state_dict(),
        }
        torch.save(
            obj=snapshot, 
            f=snapshot_full_path
        )

    def train(self, loader: data.DataLoader):
        pass
        
    def evaluate(self, loader: data.DataLoader):
        pass
    

CPU times: user 59 µs, sys: 20 µs, total: 79 µs
Wall time: 68.2 µs


In [ ]:
%%time

trainer = TuningPipeline(
    network=network,
    loss_function=loss_function,
    eval_metric=eval_metric,
    max_epochs=max_epochs,
    early_stopper=early_stopper,
    early_dataset=early_dataset,
    device=device
)

In [ ]:
train_loss = trainer.train(train_loader)

In [ ]:
print('loss on fine tuning dataset: %s ' % train_loss)

In [ ]:
eval_metric = trainer.evaluate(val_loader)

In [ ]:
print('evaluation metric on validation dataset: %s ' % eval_metric)

# Explaining MTCNN predictions manually

In [ ]:
fg, ax = plt.subplots(ncols=2, nrows=3)

for idx in range(6):
    
    img, _ = validation_dataset[idx]
    output_img = img.copy()
    boxes = trainer.predict(img)
    
    for box in range(len(boxes)):
        x1, y1, x2, y2 = box
        cv2.rectangle(img, (x1, y1), (x2, y2), color=(255,0,0), thickness=2)
    ax[idx, 0].imshow(img)
    ax[idx, 1].imshow(output_img)
plt.imshow()